## Topic Modeling (LDA) ##

### LDA :- Latent Dirichlet Allocation Model/Algorithm ###

#### By Vedant Lotia ####

In [1]:
import re
import pandas as pd
import genesis
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis



In [2]:
papers = pd.read_csv('C:\\Users\\HP\\Downloads\\papers\\papers.csv')
papers.head(10)

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."
5,1002,1994,Using a neural net to instantiate a deformable...,NaN,1002-using-a-neural-net-to-instantiate-a-defor...,Abstract Missing,U sing a neural net to instantiate a\ndeformab...
6,1003,1994,Plasticity-Mediated Competitive Learning,NaN,1003-plasticity-mediated-competitive-learning.pdf,Abstract Missing,Plasticity-Mediated Competitive Learning\n\nTe...
7,1004,1994,ICEG Morphology Classification using an Analog...,NaN,1004-iceg-morphology-classification-using-an-a...,Abstract Missing,ICEG Morphology Classification using an\nAnalo...
8,1005,1994,Real-Time Control of a Tokamak Plasma Using Ne...,NaN,1005-real-time-control-of-a-tokamak-plasma-usi...,Abstract Missing,Real-Time Control of a Tokamak Plasma\nUsing N...
9,1006,1994,Pulsestream Synapses with Non-Volatile Analogu...,NaN,1006-pulsestream-synapses-with-non-volatile-an...,Abstract Missing,Real-Time Control of a Tokamak Plasma\nUsing N...


In [3]:
papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1).sample(100)

# Print out the first rows of papers
papers.head()

,year,title,abstract,paper_text
635,1998,Classification in Non-Metric Spaces,Abstract Missing,Classification in Non-Metric Spaces\n\nDaphna ...
869,2000,A Gradient-Based Boosting Algorithm for Regres...,Abstract Missing,A Gradient-Based Boosting Algorithm for\nRegre...
2290,2006,TrueSkill?: A Bayesian Skill Rating System,Abstract Missing,TrueSkill\n\nTM :\n\nA Bayesian Skill Rating S...
2438,2007,Using Deep Belief Nets to Learn Covariance Ker...,We show how to use unlabeled data and a deep b...,Using Deep Belief Nets to Learn Covariance Ker...
442,1997,The Rectified Gaussian Distribution,Abstract Missing,The Rectified Gaussian Distribution\nN. D. Soc...


In [11]:


# Remove punctuation
papers['paper_text_processed'] = \
papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
papers['paper_text_processed'] = \
papers['paper_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
papers['paper_text_processed'].head()

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
C:\Users\HP\AppData\Local\Temp/ipykernel_15456/3911549869.py:3: DeprecationWarning: invalid escape sequence \.
  papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))


635     classification in non-metric spaces\n\ndaphna ...
869     a gradient-based boosting algorithm for\nregre...
2290    trueskill\n\ntm :\n\na bayesian skill rating s...
2438    using deep belief nets to learn covariance ker...
442     the rectified gaussian distribution\nn d socci...
Name: paper_text_processed, dtype: object

In [12]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]


data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

# remove stop words
data_words = remove_stopwords(data_words)

print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['classification', 'non', 'metric', 'spaces', 'daphna', 'weinshall', 'david', 'jacobs', 'yoram', 'gdalyahu', 'nec', 'research', 'institute', 'independence', 'way', 'princeton', 'nj', 'usa', 'inst', 'computer', 'science', 'hebrew', 'university', 'jerusalem', 'jerusalem', 'israel', 'abstract', 'key', 'question', 'vision']


In [13]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 4), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 6), (11, 11), (12, 1), (13, 1), (14, 4), (15, 1), (16, 1), (17, 2), (18, 2), (19, 1), (20, 1), (21, 1), (22, 6), (23, 1), (24, 2), (25, 2), (26, 4), (27, 1), (28, 1), (29, 1)]


In [14]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.007*"model" + 0.005*"time" + 0.004*"algorithm" + 0.004*"using" + '
  '0.004*"data" + 0.004*"learning" + 0.004*"function" + 0.003*"number" + '
  '0.003*"set" + 0.003*"state"'),
 (1,
  '0.008*"algorithm" + 0.006*"learning" + 0.005*"function" + 0.005*"model" + '
  '0.004*"set" + 0.004*"data" + 0.004*"time" + 0.004*"one" + 0.004*"problem" + '
  '0.004*"distribution"'),
 (2,
  '0.006*"learning" + 0.006*"algorithm" + 0.005*"time" + 0.005*"set" + '
  '0.005*"data" + 0.005*"model" + 0.004*"function" + 0.004*"one" + '
  '0.004*"using" + 0.004*"two"'),
 (3,
  '0.006*"model" + 0.005*"learning" + 0.004*"algorithm" + 0.004*"data" + '
  '0.004*"time" + 0.004*"set" + 0.004*"function" + 0.004*"number" + '
  '0.004*"state" + 0.003*"using"'),
 (4,
  '0.005*"model" + 0.005*"algorithm" + 0.005*"learning" + 0.004*"set" + '
  '0.004*"function" + 0.004*"data" + 0.004*"one" + 0.004*"problem" + '
  '0.004*"time" + 0.003*"matrix"'),
 (5,
  '0.006*"data" + 0.006*"model" + 0.006*"algorithm" + 0.005*"lea

In [15]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

C:\Users\HP\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.000409  0.001511       1        1  16.225818
3     -0.002056 -0.002484       2        1  13.452196
6      0.008266 -0.006274       3        1  12.938194
1     -0.000375  0.008356       4        1  12.189805
0     -0.003998 -0.005554       5        1  11.060112
9     -0.002233  0.001655       6        1  10.894375
8     -0.002897 -0.002158       7        1   6.986444
2      0.005886  0.002125       8        1   6.623535
4      0.004028  0.003005       9        1   5.215843
7     -0.006213 -0.000182      10        1   4.413677, topic_info=           Term         Freq        Total Category  logprob  loglift
10    algorithm  1269.000000  1269.000000  Default  30.0000  30.0000
447     optimal   502.000000   502.000000  Default  29.0000  29.0000
572         set  1005.000000  1005.000000  Default  28.0000  28.0000
264    function  1015.000000  1015.000000  Default  27.0000  27.0000
1982        map   270.000000   270.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
996      neural    22.701568   490.130170  Topic10  -6.1449   0.0482
483     problem    24.779787   695.498200  Topic10  -6.0573  -0.2141
218       error    22.135468   477.874358  Topic10  -6.1702   0.0483
886    gaussian    22.162728   490.561352  Topic10  -6.1689   0.0233
1986     matrix    22.399294   558.815532  Topic10  -6.1583  -0.0963

[865 rows x 6 columns], token_table=       Topic      Freq Term
term                       
6272       1  0.136999   ab
6272       2  0.182666   ab
6272       3  0.273999   ab
6272       4  0.159833   ab
6272       5  0.091333   ab
...      ...       ...  ...
12060      4  0.099381  yti
12060      5  0.099381  yti
12060      6  0.099381  yti
12060      7  0.198762  yti
12060      9  0.099381  yti

[3828 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 7, 2, 1, 10, 9, 3, 5, 8])